# Iniciar los servidores
cd kafka_2.11-2.1.0/

zookeeper-server-start.sh config/zookeeper.properties

kafka-server-start.sh config/server.properties

## Primera parte

#### crear un nuevo topic 

kafka-topics.sh --zookeeper 127.0.0.1:2181 --topic text_topic --create --partitions 3 --replication-factor 1

<pre>Created topic &quot;text_topic&quot;.
</pre>

#### ¿Serías capaz de listar todos los topics existentes?

kafka-topics.sh --zookeeper 127.0.0.1:2181 --list

<pre>__consumer_offsets
firnew_topic
first_topic
text_topic
topic_app
</pre>

#### Luego en esta misma consola crearemos un producer y comenzaremos a insertar elementos en el topic recién creado

kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic text_topic

<pre>&gt;Hola mundo
&gt;hola mundo de nuevo
&gt;tercera frase de texto
&gt;otro mensaje
&gt;
</pre>

### configurar a Presto
 
 cd presto-server-0.213/

#### crear el fichero kafka.properties

cd etc/

mkdir catalog

cd catalog

vi kafka.properties
<pre><font color="#34E2E2"><b>connector.name</b></font>=<font color="#AD7FA8">kafka</font>
<font color="#34E2E2"><b>kafka.nodes</b></font>=<font color="#AD7FA8">localhost:9092</font>
<font color="#34E2E2"><b>kafka.table-names</b></font>=<font color="#AD7FA8">text_topic</font>
<font color="#34E2E2"><b>kafka.hide-internal-columns</b></font>=<font color="#AD7FA8">false</font>

:wq</pre>

#### arrancamos el servidor de presto

bin/launcher start

<pre>Started as 13234
</pre>

### lanzamos presto

cd $HOME

./presto --catalog kafka --schema default

<pre>presto:default&gt; 
</pre>

#### Lista las tablas existente

<pre>presto:default&gt; SHOW TABLES;
   Table    
------------
 text_topic 
(1 row)

Query 20220530_105406_00001_g99mb, FINISHED, 1 node
Splits: 19 total, 19 done (100,00%)
0:03 [1 rows, 27B] [0 rows/s, 10B/s]
</pre>

#### el DESCRIBE únicamente se listan las columnas internas (relacionadas con kafka)

<pre>presto:default&gt; DESCRIBE text_topic;
      Column       |  Type   | Extra |                   Comment                   
-------------------+---------+-------+---------------------------------------------
 _partition_id     | bigint  |       | Partition Id                                
 _partition_offset | bigint  |       | Offset for the message within the partition 
 _segment_start    | bigint  |       | Segment start offset                        
 _segment_end      | bigint  |       | Segment end offset                          
 _segment_count    | bigint  |       | Running message count per segment           
 _message_corrupt  | boolean |       | Message data is corrupt                     
 _message          | varchar |       | Message text                                
 _message_length   | bigint  |       | Total number of message bytes               
 _key_corrupt      | boolean |       | Key data is corrupt                         
 _key              | varchar |       | Key text                                    
 _key_length       | bigint  |       | Total number of key bytes                   
(11 rows)

Query 20220530_105653_00003_g99mb, FINISHED, 1 node
Splits: 19 total, 19 done (100,00%)
0:00 [11 rows, 1,04KB] [27 rows/s, 2,57KB/s]
</pre>

#### Consultamos el topic y vemos los mensajes

<pre>presto:default&gt; SELECT _message FROM text_topic;
        _message        
------------------------
 tercera frase de texto 
 Hola mundo             
 otro mensaje           
 hola mundo de nuevo    
(4 rows)

Query 20220530_110649_00006_g99mb, FINISHED, 1 node
Splits: 19 total, 19 done (100,00%)
0:01 [4 rows, 63B] [4 rows/s, 73B/s]
</pre>

#### Desde la consola donde tenemos el producer introduciremos más texto 
<pre>&gt;Hola mundo
&gt;hola mundo de nuevo
&gt;tercera frase de texto
&gt;otro mensaje
&gt;mensaje para presto
&gt;
</pre>

#### Volvemos a consultar el topic

<pre>presto:default&gt; SELECT _message FROM text_topic;
        _message        
------------------------
 tercera frase de texto 
 Hola mundo             
 otro mensaje           
 hola mundo de nuevo    
 mensaje para presto    
(5 rows)

Query 20220530_111055_00007_g99mb, FINISHED, 1 node
Splits: 19 total, 19 done (100,00%)
0:00 [5 rows, 82B] [16 rows/s, 272B/s]
</pre>

## Segunda parte

### crear un nuevo topic cuyos datos serán transmitidos en formato csv

kafka-topics.sh --zookeeper 127.0.0.1:2181 --topic csv_topic --crea--partitions 3 --replication-factor 1

<pre>Created topic &quot;csv_topic&quot;.</pre>

kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic csv_topic

<pre>&gt;&quot;Pepe&quot;,&quot;20&quot;,&quot;70.5&quot;,&quot;C/Toledo 50, 1-C&quot;
&gt;</pre>

### detendremos el servicio

bin/launcher stop

<pre>Stopped 14034</pre>

### Editamos el fichero kafka.properties

<pre><font color="#34E2E2"><b>connector.name</b></font>=<font color="#AD7FA8">kafka</font>
<font color="#34E2E2"><b>kafka.nodes</b></font>=<font color="#AD7FA8">localhost:9092</font>
<font color="#34E2E2"><b>kafka.table-names</b></font>=<font color="#AD7FA8">text_topic,csv_topic</font>
<font color="#34E2E2"><b>kafka.hide-internal-columns</b></font>=<font color="#AD7FA8">false</font>
</pre>

### Iniciamos el servidor de presto

bin/launcher start

<pre>Started as 15546</pre>

### a donde tenemos iniciado el cliente de consultas de linea de comandos de presto ejecutamos las siguientes sentencias:

<pre>presto:default&gt; SHOW TABLES;
   Table    
------------
 csv_topic  
 text_topic 
(2 rows)

Query 20220530_114657_00000_6risi, FINISHED, 1 node
Splits: 19 total, 19 done (100,00%)
0:03 [2 rows, 53B] [0 rows/s, 20B/s]
</pre>

<pre>presto:default&gt; DESCRIBE csv_topic;
      Column       |  Type   | Extra |                   Comment                   
-------------------+---------+-------+---------------------------------------------
 _partition_id     | bigint  |       | Partition Id                                
 _partition_offset | bigint  |       | Offset for the message within the partition 
 _segment_start    | bigint  |       | Segment start offset                        
 _segment_end      | bigint  |       | Segment end offset                          
 _segment_count    | bigint  |       | Running message count per segment           
 _message_corrupt  | boolean |       | Message data is corrupt                     
 _message          | varchar |       | Message text                                
 _message_length   | bigint  |       | Total number of message bytes               
 _key_corrupt      | boolean |       | Key data is corrupt                         
 _key              | varchar |       | Key text                                    
 _key_length       | bigint  |       | Total number of key bytes                   
(11 rows)

Query 20220530_114737_00001_6risi, FINISHED, 1 node
Splits: 19 total, 19 done (100,00%)
0:01 [11 rows, 1,03KB] [16 rows/s, 1,52KB/s]
</pre>

### Consultamos los mensajes del topic

<pre>presto:default&gt; SELECT _message FROM csv_topic;
               _message                
---------------------------------------
 &quot;Pepe&quot;,&quot;20&quot;,&quot;70.5&quot;,&quot;C/Toledo 50, 1-C&quot; 
(1 row)

Query 20220530_114926_00002_6risi, FINISHED, 1 node
Splits: 17 total, 17 done (100,00%)
0:01 [1 rows, 37B] [1 rows/s, 51B/s]</pre>

### crear un nuevo fichero JSON con la estructura de la tabla
#### Detener el servidor presto
bin/launcher stop

mkdir etc/kafka

nano csv_topic.json 

<pre><font color="#EF2929"><b>{</b></font>
        <font color="#729FCF"><b>&quot;tableName&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#AD7FA8"><b> &quot;csv_topic&quot;</b></font><font color="#EF2929"><b>,</b></font>
        <font color="#729FCF"><b>&quot;topicName&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#AD7FA8"><b> &quot;csv_topic&quot;</b></font><font color="#EF2929"><b>,</b></font>
        <font color="#729FCF"><b>&quot;schemaName&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#AD7FA8"><b> &quot;default&quot;</b></font><font color="#EF2929"><b>,</b></font>
        <font color="#729FCF"><b>&quot;message&quot;</b></font><font color="#EF2929"><b>:</b></font> <font color="#EF2929"><b>{</b></font>
                <font color="#729FCF"><b>&quot;dataFormat&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#AD7FA8"><b>&quot;csv&quot;</b></font><font color="#EF2929"><b>,</b></font>
                 <font color="#729FCF"><b>&quot;fields&quot;</b></font><font color="#EF2929"><b>:</b></font> <font color="#729FCF"><b>[</b></font>
                        <font color="#EF2929"><b>{</b></font>
                                 <font color="#729FCF"><b>&quot;name&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#AD7FA8"><b> &quot;nombre&quot;</b></font><font color="#EF2929"><b>,</b></font>
                                 <font color="#729FCF"><b>&quot;mapping&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#4E9A06"> 0</font><font color="#EF2929"><b>,</b></font>
                                 <font color="#729FCF"><b>&quot;type&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#AD7FA8"><b> &quot;VARCHAR&quot;</b></font>
                        <font color="#EF2929"><b>},</b></font>
                        <font color="#EF2929"><b>{</b></font>
                                 <font color="#729FCF"><b>&quot;name&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#AD7FA8"><b> &quot;edad&quot;</b></font><font color="#EF2929"><b>,</b></font>
                                 <font color="#729FCF"><b>&quot;mapping&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#4E9A06"> 1</font><font color="#EF2929"><b>,</b></font>
                                 <font color="#729FCF"><b>&quot;type&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#AD7FA8"><b> &quot;INT&quot;</b></font>
                        <font color="#EF2929"><b>},</b></font>
                        <font color="#EF2929"><b>{</b></font>
                                 <font color="#729FCF"><b>&quot;name&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#AD7FA8"><b> &quot;peso&quot;</b></font><font color="#EF2929"><b>,</b></font>
                                 <font color="#729FCF"><b>&quot;mapping&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#4E9A06"> 2</font><font color="#EF2929"><b>,</b></font>
                                 <font color="#729FCF"><b>&quot;type&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#AD7FA8"><b> &quot;DOUBLE&quot;</b></font>
                         <font color="#EF2929"><b>},</b></font>
                         <font color="#EF2929"><b>{</b></font>
                                 <font color="#729FCF"><b>&quot;name&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#AD7FA8"><b> &quot;direccion&quot;</b></font><font color="#EF2929"><b>,</b></font>
                                 <font color="#729FCF"><b>&quot;mapping&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#4E9A06"> 3</font><font color="#EF2929"><b>,</b></font>
                                 <font color="#729FCF"><b>&quot;type&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#AD7FA8"><b> &quot;VARCHAR&quot;</b></font>
                         <font color="#EF2929"><b>}</b></font>
                 <font color="#729FCF"><b>]</b></font>
         <font color="#EF2929"><b>}</b></font>
<font color="#EF2929"><b>}</b></font>
</pre>

### procedemos a reiniciar de nuevo el servidor presto

bin/launcher start

### mediante presto veremos las características de la tabla y también los nuevos campos definidos en el fichero csv_topic.json

<pre>presto:default&gt; DESCRIBE csv_topic;
      Column       |  Type   | Extra |                   Comment                   
-------------------+---------+-------+---------------------------------------------
 nombre            | varchar |       |                                             
 edad              | integer |       |                                             
 peso              | double  |       |                                             
 direccion         | varchar |       |                                             
 _partition_id     | bigint  |       | Partition Id                                
 _partition_offset | bigint  |       | Offset for the message within the partition 
 _segment_start    | bigint  |       | Segment start offset                        
 _segment_end      | bigint  |       | Segment end offset                          
 _segment_count    | bigint  |       | Running message count per segment           
 _message_corrupt  | boolean |       | Message data is corrupt                     
 _message          | varchar |       | Message text                                
 _message_length   | bigint  |       | Total number of message bytes               
 _key_corrupt      | boolean |       | Key data is corrupt                         
 _key              | varchar |       | Key text                                    
 _key_length       | bigint  |       | Total number of key bytes                   
(15 rows)

Query 20220530_140256_00000_22ga6, FINISHED, 1 node
Splits: 19 total, 19 done (100,00%)
0:03 [15 rows, 1,29KB] [5 rows/s, 465B/s]
</pre>

<pre>presto:default&gt; select * from csv_topic;
 nombre | edad | peso |    direccion     | _partition_id | _partition_offset | _segment_start | 
--------+------+------+------------------+---------------+-------------------+----------------+-
 Pepe   |   20 | 70.5 | C/Toledo 50, 1-C |             2 |                 0 |              0 | </pre>
 
 <pre>presto:default&gt; SELECT nombre, edad, peso, direccion FROM csv_topic
             -&gt; where edad = 20;
 nombre | edad | peso |    direccion     
--------+------+------+------------------
 Pepe   |   20 | 70.5 | C/Toledo 50, 1-C 
(1 row)

Query 20220530_141756_00003_22ga6, FINISHED, 1 node
Splits: 17 total, 17 done (100,00%)
0:00 [1 rows, 37B] [2 rows/s, 92B/s]</pre>

## Tercera parte

### crearemos un nuevo topic denominado json_topic

kafka-topics.sh --zookeeper 127.0.0.1:2181 --topic json_topic --create --partitions 3 --replication-factor 1

### insertamos un registro en el topic 

 siempre 1 registro se corresponde a una linea
 
 kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic json_topic
 
 Mal escrito:
 <pre>&gt;{&quot;nombre&quot;:&quot;JOSE ANTONIO&quot;,&quot;edad&quot;:38,&quot;peso&quot;:75.5,&quot;direccion&quot;,&quot;C/ DEL LIMONERO 39, PISO 1-A&quot;}
&gt;
</pre>
Bien escrito:
 <pre>&gt;{&quot;nombre&quot;:&quot;JOSE ANTONIO&quot;,&quot;edad&quot;:38,&quot;peso&quot;:75.5,&quot;direccion&quot;:&quot;C/ DEL LIMONERO 39, PISO 1-A&quot;}
&gt;
</pre>

### detendremos el servicio y actualizaremos el fichero de kafka para añadir una nueva tabla

bin/launcher stop

cd catalog

nano kafka.properties

<pre>connector.name=kafka
kafka.nodes=localhost:9092
kafka.table-names=text_topic,csv_topic,json_topic
kafka.hide-internal-columns=false
</pre>

###  añadiremos la definición de la tabla

<pre><font color="#8AE234"><b>spark@spark-virtualBox</b></font>:<font color="#729FCF"><b>~/presto-server-0.213/etc</b></font>$ ls
<font color="#729FCF"><b>catalog</b></font>  config.properties  jvm.config  <font color="#729FCF"><b>kafka</b></font>  log.properties  node.properties</pre>

cd kafka

nano json_topic.json

<pre><font color="#EF2929"><b>{</b></font>
        <font color="#729FCF"><b>&quot;tableName&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#AD7FA8"><b> &quot;json_topic&quot;</b></font><font color="#EF2929"><b>,</b></font>
        <font color="#729FCF"><b>&quot;topicName&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#AD7FA8"><b> &quot;json_topic&quot;</b></font><font color="#EF2929"><b>,</b></font>
        <font color="#729FCF"><b>&quot;schemaName&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#AD7FA8"><b> &quot;default&quot;</b></font><font color="#EF2929"><b>,</b></font>
        <font color="#729FCF"><b>&quot;message&quot;</b></font><font color="#EF2929"><b>:</b></font> <font color="#EF2929"><b>{</b></font>
                <font color="#729FCF"><b>&quot;dataFormat&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#AD7FA8"><b>&quot;json&quot;</b></font><font color="#EF2929"><b>,</b></font>
                <font color="#729FCF"><b>&quot;fields&quot;</b></font><font color="#EF2929"><b>:</b></font> <font color="#729FCF"><b>[</b></font>
                        <font color="#EF2929"><b>{</b></font>
                                <font color="#729FCF"><b>&quot;name&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#AD7FA8"><b> &quot;nombre&quot;</b></font><font color="#EF2929"><b>,</b></font>
                                <font color="#729FCF"><b>&quot;mapping&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#AD7FA8"><b> &quot;nombre&quot;</b></font><font color="#EF2929"><b>,</b></font>
                                <font color="#729FCF"><b>&quot;type&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#AD7FA8"><b> &quot;VARCHAR&quot;</b></font>
                        <font color="#EF2929"><b>},</b></font>
                        <font color="#EF2929"><b>{</b></font>
                                <font color="#729FCF"><b>&quot;name&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#AD7FA8"><b> &quot;edad&quot;</b></font><font color="#EF2929"><b>,</b></font>
                                <font color="#729FCF"><b>&quot;mapping&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#AD7FA8"><b> &quot;edad&quot;</b></font><font color="#EF2929"><b>,</b></font>
                                <font color="#729FCF"><b>&quot;type&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#AD7FA8"><b> &quot;INT&quot;</b></font>
                        <font color="#EF2929"><b>},</b></font>
                        <font color="#EF2929"><b>{</b></font>
                                <font color="#729FCF"><b>&quot;name&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#AD7FA8"><b> &quot;peso&quot;</b></font><font color="#EF2929"><b>,</b></font>
                                <font color="#729FCF"><b>&quot;mapping&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#AD7FA8"><b> &quot;peso&quot;</b></font><font color="#EF2929"><b>,</b></font>
                                <font color="#729FCF"><b>&quot;type&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#AD7FA8"><b> &quot;DOUBLE&quot;</b></font>
                        <font color="#EF2929"><b>},</b></font>
                        <font color="#EF2929"><b>{</b></font>
                                <font color="#729FCF"><b>&quot;name&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#AD7FA8"><b> &quot;direccion&quot;</b></font><font color="#EF2929"><b>,</b></font>
                                <font color="#729FCF"><b>&quot;mapping&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#AD7FA8"><b> &quot;direccion&quot;</b></font><font color="#EF2929"><b>,</b></font>
                                <font color="#729FCF"><b>&quot;type&quot;</b></font><font color="#EF2929"><b>:</b></font><font color="#AD7FA8"><b> &quot;VARCHAR&quot;</b></font>
                        <font color="#EF2929"><b>}</b></font>
                <font color="#729FCF"><b>]</b></font>
        <font color="#EF2929"><b>}</b></font>
<font color="#EF2929"><b>}</b></font>
</pre>

### reiniciar de nuevo el servidor presto

cd presto-server-0.213

bin/launcher start

### desde presto observar las caracteristicas de la tabla 

<pre>presto:default&gt; show tables;
   Table    
------------
 csv_topic  
 json_topic 
 text_topic 
(3 rows)

Query 20220530_151722_00000_xkw5f, FINISHED, 1 node
Splits: 19 total, 19 done (100,00%)
0:03 [3 rows, 80B] [1 rows/s, 28B/s]</pre>

<pre>presto:default&gt; describe json_topic;
      Column       |  Type   | Extra |                   Comment                   
-------------------+---------+-------+---------------------------------------------
 nombre            | varchar |       |                                             
 edad              | integer |       |                                             
 peso              | double  |       |                                             
 direccion         | varchar |       |                                             
 _partition_id     | bigint  |       | Partition Id                                
 _partition_offset | bigint  |       | Offset for the message within the partition 
 _segment_start    | bigint  |       | Segment start offset                        
 _segment_end      | bigint  |       | Segment end offset                          
 _segment_count    | bigint  |       | Running message count per segment           
 _message_corrupt  | boolean |       | Message data is corrupt                     
 _message          | varchar |       | Message text                                
 _message_length   | bigint  |       | Total number of message bytes               
 _key_corrupt      | boolean |       | Key data is corrupt                         
 _key              | varchar |       | Key text                                    
 _key_length       | bigint  |       | Total number of key bytes                   
(15 rows)

Query 20220530_151803_00001_xkw5f, FINISHED, 1 node
Splits: 19 total, 19 done (100,00%)
0:01 [15 rows, 1,31KB] [22 rows/s, 1,93KB/s]
</pre>

<pre>presto:default&gt; select * from json_topic;
    nombre    | edad | peso |          direccion           | _partition_id | _partition_offset 
--------------+------+------+------------------------------+---------------+-------------------
 NULL         | NULL | NULL | NULL                         |             1 |                 0 
 NULL         | NULL | NULL | NULL                         |             0 |                 0 
 JOSE ANTONIO |   38 | 75.5 | C/ DEL LIMONERO 39, PISO 1-A |             2 |                 0 
(3 rows)
</pre>

<pre>presto:default&gt; select nombre, edad, peso, direccion from json_topic where edad = 38;
    nombre    | edad | peso |          direccion           
--------------+------+------+------------------------------
 JOSE ANTONIO |   38 | 75.5 | C/ DEL LIMONERO 39, PISO 1-A 
(1 row)

Query 20220601_072629_00003_3ycep, FINISHED, 1 node
Splits: 19 total, 19 done (100,00%)
0:01 [3 rows, 252B] [4 rows/s, 338B/s]
</pre>